# 機械学習の数学 4（単回帰分析）

5 章では機械学習の入口として最も簡単なアルゴリズムの 1 つである単回帰分析について解説します。  
理論背景がシンプルなので機械学習の大枠を掴む上で最適なテーマとなっています。  

単回帰分析は 1 つの入力変数から 1 つの出力変数を予測する問題設定で使用します。  
例えば部屋の広さを入力変数 $x$ として、家賃を出力変数 $y$ として予測するような問題設定です。  

本章で扱う単回帰分析の一連手順はディープラーニングを始めとした機械学習全般に共通する基礎知識となります。  
以降の章で扱う内容のにも大きく関係するので、応用することを前提として考え方の枠組を掴むことを特に意識してください。

## 本章の構成
- モデルを決める
- 目的関数を決める
- 最適なパラメータを求める
- 予測値を求める

## モデルを決める

### 定式化

まずはじめに、入力変数 $x$ と出力変数 $y$ との関係をどのように定式化するかを決定します。この定式化したものを**モデル**もしくは**数理モデル**と呼びます。機械学習というとデータに基づいてその規則性を見つけるといった印象が強いのですが、この最初の定式化は人間が考える必要があり、データに基づいて決定されるものではありません。定式化の種類はたくさんあるため、機械学習を学びながら徐々に引き出しを増やしていってください。

例えば、**部屋の広さ**と**家賃**に関するデータを取得しておき、その関係性を以下の図のようにプロットしてみます。

![01](http://drive.google.com/uc?export=view&id=12eLqQKm-91sxtJH8FcXh8n9P-ewQq1vi)

そして、このデータを関係性をうまく表現できるような定式化を考えます。最も簡単な定式化として、中学校のときに習った一次関数 $y=wx+b$ があります。中学校のときには $ax+b$ でしたが、機械学習では $wx+b$ と記述することが一般的です。

![02](http://drive.google.com/uc?export=view&id=1wtbtocjXo3n4dgf9kK31bG_i7-pyV2o1)

この定式化で完璧に表現できているというわけではありませんが、ある程度うまく表現できそうだと言うことがわかります。データに基づいて規則性を見つけられるのであれば、予測に誤りがない完璧なモデルができると思われがちですが、家賃のようにすべての物件の金額を網羅できるような規則性がない場合もあるため気をつけましょう。また、このときのモデルをうまく選べないと、どれだけデータを集めてもうまく予測できるモデルを構築することができません。

それでは、$y=wx+b$ と定式化したとき、次に何を調整するべきか考えます。データが取得できている前提で考えると、部屋の広さ $x$ と家賃 $y$ はそれぞれ取得できています。それに対し、直線の傾き $w$ と切片 $b$ はこれから調整すべきものであることがわかります。この $w$ や $b$ のようにデータに基づいて調整する変数を**パラメータ**と呼び、今後頻出する単語であるため覚えておきましょう。機械学習とは定式化したパラメータを推定する問題と考えることができます。また、$w$ と $b$ も今後頻出し、$w$ は**重み (weight)**, $b$ は**バイアス (bias)** の頭文字です。 

### 前処理

次のステップに進む前に、データの**前処理 (pre-processing)** のひとつであるデータの **中心化 (centering)** を紹介します。この前処理によって、これから進める計算が楽になります。
データの中心化は、以下の図のようにデータの平均値が 0 になるように全てのデータを移動する処理です。

![03](http://drive.google.com/uc?export=view&id=1q2t_EZTEqufWQXnjEmFXNLUKkfwlmWhl)

中心化の処理は簡単で、各変数ごとに平均を求め、すべてのデータに対して、その平均を引きます。例えば、入力値 $x_{n}$, 目標値 $t_{n}$ $(n=1,2,\ldots, N)$ が与えられた場合、その平均 $\bar{x}$, $\bar{t}$ は以下のようにすべてを足して、その数で割れば求まります。

$$
\begin{aligned}
\bar{x}
&= \dfrac{1}{N}\left(x_{1} + x_{2} + \dots + x_{N}\right)
= \dfrac{1}{N} \sum_{n=1}^{N}x_{n} \\
\bar{t}
&= \dfrac{1}{N}\left(t_{1} + t_{2} + \dots + t_{N}\right)
= \dfrac{1}{N} \sum_{n=1}^{N}t_{n} \\
\end{aligned}
$$

この平均値を用いて、中心化後の変数は以下のようになります。

$$
\begin{aligned}
x_{c} &= x - \bar{x} \\
t_{c} &= t - \bar{t}
\end{aligned}
$$

この中心化後には切片が 0 となるため、$b$ のパラメータを省略することができ、パラメータの数が 1 つになり、計算を少なくすることができます。

$$
y_{c} = wx_{c}
$$

以降は記述を簡単にするため、$_c$ という添え字を省略し、事前に中心化を行っている前提でデータを扱います。また、そのデータにフィットさせたいモデルは、

$$
y = wx
$$

と、こちらも添え字 $_c$ を省略して説明を行います。

## 目的関数を決める

機械学習では「最適なパラメータを求める」という問題へと落としこむことができましたが、「最適なパラメータ」の定義を決めないと次に進むことができません。そこで、評価の指標とする関数のことを**目的関数**と呼び、本資料では $L$ を文字として使います。分野によっては**評価関数**と呼ぶこともあります。今回の問題設定では、どのように指標を決めると良いでしょうか。すでに微分のときに紹介している通り、目標値と予測値の**誤差** $t-y$ を最小化することが望ましいと言えます。ただし、単に $t-y$ としてしまうと、マイナス方向にいくらでも小さくできてしまうため、$|t-y|$ のように絶対値を取るか、$(t-y)^{2}$ のように二乗とします。どちらも指標として使えますが、計算の特性上、二乗を採用することが一般的であり、単回帰分析もこの二乗誤差を採用します。

$n$ 個目のデータの部屋の広さ $x_n$ が与えられたときのモデルの予測値 $y_n$ と、対応する目標値 $t_n$ との間の二乗誤差は、

$$
(t_{n} - y_{n})^{2}
$$

となります。これを全データに渡って合計したものが以下の**二乗和誤差**であり、単回帰分析の目的関数として採用します。

$$
\begin{aligned}
L
&= (t_1 - y_1)^2
+ (t_2 - y_2)^2
+ \cdots
+ (t_N - y_N)^2 \\
&= \sum_{n=1}^N (t_n - y_n )^2 \\
\end{aligned}
$$

## 最適なパラメータを求める

採用した目的関数を最小化するようなパラメータを求めます。データの中心化を前提とした単回帰分析のパラメータは $w$ でした。そして、微分のときに学んだ通り、誤差が最小となる点は「接線の傾きが 0 」となるように決定します。

それでは、具体的に今回定めた目的関数 $L$ をパラメータである $w$ で微分して導関数を求め、その微分係数が 0 となるような $w$ を求めます。$y=wx$ という関係と微分の線形性を利用して展開を進めると、

$$
\begin{aligned}
\frac{\partial}{\partial w} L
&= \frac{\partial}{\partial w} \sum_{n=1}^N (t_n - y_n)^2 \\
&= \frac{\partial}{\partial w} \sum_{n=1}^N (t_n - wx_n)^2 \\
&= \frac{\partial}{\partial w} \left\{ (t_1 - wx_1)^2 + (t_2 - wx_2)^2 + \dots + (t_N - wx_N)^2\right\} \\
&= \frac{\partial}{\partial w} (t_1 - wx_1)^2 + \frac{\partial}{\partial w}(t_2 - wx_2)^2 + \dots + \frac{\partial}{\partial w}(t_N - wx_N)^2 \\
&= \sum_{n=1}^N \frac{\partial}{\partial w} (t_n - wx_n)^2
\end{aligned}
$$

となります。つぎに、総和の各項目である $\frac{\partial}{\partial w} (t_n - wx_n)^2$ の計算について着目すると、合成関数になっており、以下のように分解できます。

$$
\begin{aligned}
u(w) &= t_{n} - wx_{n} \\
f(u) &= u^{2}
\end{aligned}
$$

それぞれの微分は

$$
\begin{aligned}
\dfrac{\partial u(w)}{\partial w} &= \dfrac{\partial u}{\partial w} = -x_{n} \\
\dfrac{\partial f(u)}{\partial u} &= \dfrac{\partial f}{\partial u} = 2 u
\end{aligned}
$$

となります。そして、連鎖律を用いて合成関数の微分を行うと、

$$
\begin{aligned}
\frac{\partial}{\partial w} (t_n - wx_n)^2
&= \frac{\partial}{\partial w} f(u) \\
&= \frac{\partial u}{\partial w}\frac{\partial f}{\partial u} \\
&= -x_{n} (2u) \\
&= -2x_{n}(t_{n}-y_{n})　
\end{aligned}
$$

が得られます。これより、目的関数の導関数は、

$$
\frac{\partial}{\partial w} L
= -2 \sum_{n=1}^N x_{n}(t_{n}-y_{n})
$$

となります。この導関数の微分係数を 0 と置いたときのパラメータ $w$ を求めましょう。

$$
\begin{aligned}
\frac{\partial}{\partial w} L &= 0 \\
-2 \sum_{n=1}^N x_n (t_n - wx_n) &= 0 \\
-2 \sum_{n=1}^N x_n t_n + 2 \sum_{n=1}^N wx^2_n &= 0 \\
-2 \sum_{n=1}^N x_n t_n + 2 w \sum_{n=1}^N x^2_n &= 0 \\
w \sum_{n=1}^N x^2_n &= \sum_{n=1}^N x_n t_n \\
w &= \frac{\sum_{n=1}^N x_n t_n}{\sum_{n=1}^N x^2_n}
\end{aligned}
$$

このように、単回帰分析のパラメータを与えられたデータ $x_{n}$, $t_{n}$ $(n=1,2,\dots,N)$ によって決定することができています。

## 予測値を求める

データに基づいてパラメータ $w$ が決定しているとき、新しい入力値 $x$ が与えられた場合に、予測値 $y$ の計算の流れは以下のとおりです。

$$
\begin{aligned}
y_c &= wx_c \\
y - \bar{t} &= w(x - \bar{x}) \\
y &= w(x - \bar{x}) + \bar{t}
\end{aligned}
$$

事前にデータの中心化を行っていたため、予測を行うときにはその点も考慮して計算する必要があることがわかります。

それでは、簡単な例題で計算してみましょう。以下のようなデータが与えられており、$x=3$ での家賃の予測値 $y$ を求めたいとします。

|部屋の広さ|家賃|
|:--|:--|
|2|8|
|3|?|
|4|12|
|6|16|

この問題は計算しやすいようにある直線を引けば全く誤差が出ないように設定しており、勘の良い方だと計算するまでもなく、$y=2x+4$ であることがわかると思います。したがって、うまく求めることができれば $y=10$ となります。

それでは、与えられた入力値と目標値を使って、パラメータ w を求めましょう。まずはデータの中心化を行います。

$$
\begin{aligned}
\bar{x} &= \dfrac{2 + 4 + 6}{3} = 4 \\
\bar{t} &= \dfrac{8 + 12 + 16}{3} = 12 \\
\end{aligned}
$$

したがって、中心化後のデータは

|$x_{c}$|$t_{c}$|
|:--|:--|
|-2|-4|
|0|0|
|2|4|

となります。これらの値を用いてパラメータを求めます。

$$
\begin{aligned}
w &= \dfrac{\sum_{n=1}^N x_n t_n}{\sum_{n=1}^N x^2_n} \\
&= \dfrac{(-2)\times (-4) + 0\times0 + 2 \times 4}{(-2)^{2} + 0^{2} + 2^{2}} \\
&= 2
\end{aligned}
$$

これより、予測値を計算すると、

$$
\begin{aligned}
y &= w(x - \bar{x}) + \bar{t} \\
&= 2(3 - 4)) + 12 \\
&=10
\end{aligned}
$$

となり、先述した理想的な予測値と一致していることがわかります。実際にはこの計算よりもサンプル数が多く、さらにはすべてのデータが同一直線上にすべて乗っているということはありませんが、どのように計算していくかの流れがつかめたと思います。

## 練習問題 本章のまとめ
本章で学んだ内容を復習しましょう。問題の答えをそれぞれのセルに記載してください。  
（プログラミングを行うのではなく、問題をノート上などで解き、回答を空白のセルに記入してください。）

単回帰分析を用いて、下記の数値を用いて ? の値求めてください。


|部屋の広さ|家賃|
|:--|:--|
|2|6|
|3|?|
|6|8|
|10|16|

問 1. 中心化後の値を算出してください。

In [ ]:
# 問 1. の計算結果


問 2. 中心化後の値を用いてパラメータ $w$ を求めてください。

In [ ]:
# 問 2. の計算結果


問 3. パラメータ $w$ を用いて、? に該当する予測値を算出してください。

In [ ]:
# 問 3. の計算結果


<img src="http://drive.google.com/uc?export=view&id=1g2xjXbw5qYeqdJqcOf3uASvzBQxhlE8u" width=30%>